# Experiment 001: Baseline with C++ Optimizer

This notebook implements the baseline using the C++ tree packer optimizer from the santa-claude kernel.

**Strategy:**
1. Compile the C++ optimizer
2. Run optimization on sample_submission.csv
3. Apply fix_direction rotation optimization
4. Validate and score the result

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
from shapely.ops import unary_union
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import os
import subprocess

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal("1e18")

print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
# Define the ChristmasTree class
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        # Define the 15 vertices of the tree polygon
        initial_polygon = Polygon(
            [
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        
        # Apply rotation and translation to the polygon
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated, 
            xoff=float(self.center_x * scale_factor), 
            yoff=float(self.center_y * scale_factor)
        )

    def clone(self):
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
# Helper functions for loading, scoring, and validation

def load_configuration_from_df(n: int, df: pd.DataFrame) -> list:
    """Loads all trees for a given N from the submission DataFrame."""
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:] if str(row["x"]).startswith('s') else str(row["x"])
        y = str(row["y"])[1:] if str(row["y"]).startswith('s') else str(row["y"])
        deg = str(row["deg"])[1:] if str(row["deg"]).startswith('s') else str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees


def get_score(trees: list, n: int) -> float:
    """Calculates the score (S^2 / N) for a given configuration of trees."""
    if not trees:
        return 0.0
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    side_length = max(max_x - min_x, max_y - min_y)
    return side_length**2 / n


def has_overlap(trees: list) -> bool:
    """Check if any two ChristmasTree polygons overlap."""
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False


def score_and_validate_submission(file_path: str, max_n: int = 200) -> dict:
    """Reads a submission CSV, calculates the total score, and checks for overlaps."""
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return {"status": "FAILED", "error": "File Not Found"}
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return {"status": "FAILED", "error": f"CSV Read Error: {e}"}

    total_score = 0.0
    failed_overlap_n = []
    
    print(f"--- Scoring and Validation: {file_path} (N=1 to {max_n}) ---")

    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        if trees:
            current_score = get_score(trees, n)
            total_score += current_score
            if has_overlap(trees):
                failed_overlap_n.append(n)
                print(f"  ❌ N={n:03d}: OVERLAP DETECTED! (Score contribution: {current_score:.6f})")
        
    print("\n--- Summary ---")
    if failed_overlap_n:
        print(f"❌ **Validation FAILED**: Overlaps found in N: {failed_overlap_n}")
        status = "FAILED (Overlaps)"
    else:
        print("✅ **Validation SUCCESSFUL**: No overlaps detected.")
        status = "SUCCESS"
        
    print(f"**Total Submission Score (Σ S²/N): {total_score:.6f}**")
    
    return {
        "status": status,
        "total_score": total_score,
        "failed_overlap_n": failed_overlap_n
    }

print("Helper functions defined")

Helper functions defined


In [4]:
# First, let's score the sample submission to get a baseline
sample_submission_path = '/home/data/sample_submission.csv'
print("Scoring sample submission...")
baseline_result = score_and_validate_submission(sample_submission_path)
print(f"\nBaseline score: {baseline_result['total_score']:.6f}")

Scoring sample submission...
--- Scoring and Validation: /home/data/sample_submission.csv (N=1 to 200) ---



--- Summary ---
✅ **Validation SUCCESSFUL**: No overlaps detected.
**Total Submission Score (Σ S²/N): 173.652299**

Baseline score: 173.652299


In [5]:
# Write the C++ optimizer code
cpp_code = '''// Tree Packer - Simplified version for baseline
// Compile: g++ -O3 -march=native -std=c++17 -fopenmp -o tree_packer tree_packer.cpp

#include <bits/stdc++.h>
#include <omp.h>
using namespace std;

constexpr int MAX_N = 200;
constexpr int NV = 15;
constexpr double PI = 3.14159265358979323846;

alignas(64) const long double TX[NV] = {0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125};
alignas(64) const long double TY[NV] = {0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5};

struct FastRNG {
    uint64_t s[2];
    FastRNG(uint64_t seed = 42) {
        s[0] = seed ^ 0x853c49e6748fea9bULL;
        s[1] = (seed * 0x9e3779b97f4a7c15ULL) ^ 0xc4ceb9fe1a85ec53ULL;
    }
    inline uint64_t rotl(uint64_t x, int k) { return (x << k) | (x >> (64 - k)); }
    inline uint64_t next() {
        uint64_t s0 = s[0], s1 = s[1], r = s0 + s1;
        s1 ^= s0; s[0] = rotl(s0, 24) ^ s1 ^ (s1 << 16); s[1] = rotl(s1, 37);
        return r;
    }
    inline long double rf() { return (next() >> 11) * 0x1.0p-53L; }
    inline long double rf2() { return rf() * 2.0L - 1.0L; }
    inline int ri(int n) { return next() % n; }
    inline long double gaussian() {
        long double u1 = rf() + 1e-10L, u2 = rf();
        return sqrtl(-2.0L * logl(u1)) * cosl(2.0L * PI * u2);
    }
};

struct Poly {
    long double px[NV], py[NV];
    long double x0, y0, x1, y1;
};

inline void getPoly(long double cx, long double cy, long double deg, Poly& q) {
    long double rad = deg * (PI / 180.0L);
    long double s = sinl(rad), c = cosl(rad);
    long double minx = 1e9L, miny = 1e9L, maxx = -1e9L, maxy = -1e9L;
    for (int i = 0; i < NV; i++) {
        long double x = TX[i] * c - TY[i] * s + cx;
        long double y = TX[i] * s + TY[i] * c + cy;
        q.px[i] = x; q.py[i] = y;
        if (x < minx) minx = x; if (x > maxx) maxx = x;
        if (y < miny) miny = y; if (y > maxy) maxy = y;
    }
    q.x0 = minx; q.y0 = miny; q.x1 = maxx; q.y1 = maxy;
}

inline bool pip(long double px, long double py, const Poly& q) {
    bool in = false;
    int j = NV - 1;
    for (int i = 0; i < NV; i++) {
        if ((q.py[i] > py) != (q.py[j] > py) &&
            px < (q.px[j] - q.px[i]) * (py - q.py[i]) / (q.py[j] - q.py[i]) + q.px[i])
            in = !in;
        j = i;
    }
    return in;
}

inline bool segInt(long double ax, long double ay, long double bx, long double by,
                   long double cx, long double cy, long double dx, long double dy) {
    long double d1 = (dx-cx)*(ay-cy) - (dy-cy)*(ax-cx);
    long double d2 = (dx-cx)*(by-cy) - (dy-cy)*(bx-cx);
    long double d3 = (bx-ax)*(cy-ay) - (by-ay)*(cx-ax);
    long double d4 = (bx-ax)*(dy-ay) - (by-ay)*(dx-ax);
    return ((d1 > 0) != (d2 > 0)) && ((d3 > 0) != (d4 > 0));
}

inline bool overlap(const Poly& a, const Poly& b) {
    if (a.x1 < b.x0 || b.x1 < a.x0 || a.y1 < b.y0 || b.y1 < a.y0) return false;
    for (int i = 0; i < NV; i++) {
        if (pip(a.px[i], a.py[i], b)) return true;
        if (pip(b.px[i], b.py[i], a)) return true;
    }
    for (int i = 0; i < NV; i++) {
        int ni = (i + 1) % NV;
        for (int j = 0; j < NV; j++) {
            int nj = (j + 1) % NV;
            if (segInt(a.px[i], a.py[i], a.px[ni], a.py[ni],
                      b.px[j], b.py[j], b.px[nj], b.py[nj])) return true;
        }
    }
    return false;
}

struct Cfg {
    int n;
    long double x[MAX_N], y[MAX_N], a[MAX_N];
    Poly pl[MAX_N];
    long double gx0, gy0, gx1, gy1;

    inline void upd(int i) { getPoly(x[i], y[i], a[i], pl[i]); }
    inline void updAll() { for (int i = 0; i < n; i++) upd(i); updGlobal(); }

    inline void updGlobal() {
        gx0 = gy0 = 1e9L; gx1 = gy1 = -1e9L;
        for (int i = 0; i < n; i++) {
            if (pl[i].x0 < gx0) gx0 = pl[i].x0;
            if (pl[i].x1 > gx1) gx1 = pl[i].x1;
            if (pl[i].y0 < gy0) gy0 = pl[i].y0;
            if (pl[i].y1 > gy1) gy1 = pl[i].y1;
        }
    }

    inline bool hasOvl(int i) const {
        for (int j = 0; j < n; j++)
            if (i != j && overlap(pl[i], pl[j])) return true;
        return false;
    }

    inline bool anyOvl() const {
        for (int i = 0; i < n; i++)
            for (int j = i + 1; j < n; j++)
                if (overlap(pl[i], pl[j])) return true;
        return false;
    }

    inline long double side() const { return max(gx1 - gx0, gy1 - gy0); }
    inline long double score() const { long double s = side(); return s * s / n; }
};

// Squeeze
Cfg squeeze(Cfg c) {
    long double cx = (c.gx0 + c.gx1) / 2.0L, cy = (c.gy0 + c.gy1) / 2.0L;
    for (long double scale = 0.9995L; scale >= 0.98L; scale -= 0.0005L) {
        Cfg trial = c;
        for (int i = 0; i < c.n; i++) {
            trial.x[i] = cx + (c.x[i] - cx) * scale;
            trial.y[i] = cy + (c.y[i] - cy) * scale;
        }
        trial.updAll();
        if (!trial.anyOvl()) c = trial;
        else break;
    }
    return c;
}

// Compaction
Cfg compaction(Cfg c, int iters) {
    long double bs = c.side();
    for (int it = 0; it < iters; it++) {
        long double cx = (c.gx0 + c.gx1) / 2.0L, cy = (c.gy0 + c.gy1) / 2.0L;
        bool improved = false;
        for (int i = 0; i < c.n; i++) {
            long double ox = c.x[i], oy = c.y[i];
            long double dx = cx - c.x[i], dy = cy - c.y[i];
            long double d = sqrtl(dx*dx + dy*dy);
            if (d < 1e-6L) continue;
            for (long double step : {0.02L, 0.008L, 0.003L, 0.001L, 0.0004L}) {
                c.x[i] = ox + dx/d * step; c.y[i] = oy + dy/d * step; c.upd(i);
                if (!c.hasOvl(i)) {
                    c.updGlobal();
                    if (c.side() < bs - 1e-12L) { bs = c.side(); improved = true; ox = c.x[i]; oy = c.y[i]; }
                    else { c.x[i] = ox; c.y[i] = oy; c.upd(i); }
                } else { c.x[i] = ox; c.y[i] = oy; c.upd(i); }
            }
        }
        c.updGlobal();
        if (!improved) break;
    }
    return c;
}

// Local search
Cfg localSearch(Cfg c, int maxIter) {
    long double bs = c.side();
    const long double steps[] = {0.01L, 0.004L, 0.0015L, 0.0006L, 0.00025L, 0.0001L};
    const long double rots[] = {5.0L, 2.0L, 0.8L, 0.3L, 0.1L};
    const int dx[] = {1,-1,0,0,1,1,-1,-1};
    const int dy[] = {0,0,1,-1,1,-1,1,-1};

    for (int iter = 0; iter < maxIter; iter++) {
        bool improved = false;
        for (int i = 0; i < c.n; i++) {
            long double cx = (c.gx0 + c.gx1) / 2.0L, cy = (c.gy0 + c.gy1) / 2.0L;
            long double ddx = cx - c.x[i], ddy = cy - c.y[i];
            long double dist = sqrtl(ddx*ddx + ddy*ddy);
            if (dist > 1e-6L) {
                for (long double st : steps) {
                    long double ox = c.x[i], oy = c.y[i];
                    c.x[i] += ddx/dist * st; c.y[i] += ddy/dist * st; c.upd(i);
                    if (!c.hasOvl(i)) { c.updGlobal(); if (c.side() < bs - 1e-12L) { bs = c.side(); improved = true; }
                        else { c.x[i]=ox; c.y[i]=oy; c.upd(i); c.updGlobal(); } }
                    else { c.x[i]=ox; c.y[i]=oy; c.upd(i); }
                }
            }
            for (long double st : steps) {
                for (int d = 0; d < 8; d++) {
                    long double ox=c.x[i], oy=c.y[i];
                    c.x[i] += dx[d]*st; c.y[i] += dy[d]*st; c.upd(i);
                    if (!c.hasOvl(i)) { c.updGlobal(); if (c.side() < bs - 1e-12L) { bs = c.side(); improved = true; }
                        else { c.x[i]=ox; c.y[i]=oy; c.upd(i); c.updGlobal(); } }
                    else { c.x[i]=ox; c.y[i]=oy; c.upd(i); }
                }
            }
            for (long double rt : rots) {
                for (long double da : {rt, -rt}) {
                    long double oa = c.a[i]; c.a[i] += da;
                    while (c.a[i] < 0) c.a[i] += 360; while (c.a[i] >= 360) c.a[i] -= 360;
                    c.upd(i);
                    if (!c.hasOvl(i)) { c.updGlobal(); if (c.side() < bs - 1e-12L) { bs = c.side(); improved = true; }
                        else { c.a[i]=oa; c.upd(i); c.updGlobal(); } }
                    else { c.a[i]=oa; c.upd(i); }
                }
            }
        }
        if (!improved) break;
    }
    return c;
}

// Simulated Annealing
Cfg simulatedAnnealing(Cfg c, int maxIter, FastRNG& rng) {
    long double T = 0.1L, Tmin = 1e-6L, alpha = 0.9995L;
    long double bs = c.side();
    Cfg best = c;
    
    for (int iter = 0; iter < maxIter && T > Tmin; iter++) {
        int i = rng.ri(c.n);
        long double ox = c.x[i], oy = c.y[i], oa = c.a[i];
        
        int moveType = rng.ri(3);
        if (moveType == 0) {
            c.x[i] += rng.gaussian() * 0.02L * T;
            c.y[i] += rng.gaussian() * 0.02L * T;
        } else if (moveType == 1) {
            c.a[i] += rng.gaussian() * 5.0L * T;
            while (c.a[i] < 0) c.a[i] += 360; while (c.a[i] >= 360) c.a[i] -= 360;
        } else {
            long double cx = (c.gx0 + c.gx1) / 2.0L, cy = (c.gy0 + c.gy1) / 2.0L;
            long double dx = cx - c.x[i], dy = cy - c.y[i];
            long double d = sqrtl(dx*dx + dy*dy);
            if (d > 1e-6L) {
                c.x[i] += dx/d * 0.01L * rng.rf();
                c.y[i] += dy/d * 0.01L * rng.rf();
            }
        }
        c.upd(i);
        
        if (c.hasOvl(i)) {
            c.x[i] = ox; c.y[i] = oy; c.a[i] = oa; c.upd(i);
        } else {
            c.updGlobal();
            long double ns = c.side();
            if (ns < bs) {
                bs = ns;
                best = c;
            } else if (rng.rf() < expl((bs - ns) / T)) {
                // Accept worse solution with probability
            } else {
                c.x[i] = ox; c.y[i] = oy; c.a[i] = oa; c.upd(i); c.updGlobal();
            }
        }
        T *= alpha;
    }
    return best;
}

Cfg configs[MAX_N + 1];
long double best_sides[MAX_N + 1];

void parse_csv(const string& filename) {
    ifstream f(filename);
    string line;
    getline(f, line); // header
    
    for (int n = 1; n <= MAX_N; n++) {
        configs[n].n = n;
        best_sides[n] = 1e9L;
    }
    
    while (getline(f, line)) {
        // Parse: id,x,y,deg
        size_t p1 = line.find(',');
        size_t p2 = line.find(',', p1+1);
        size_t p3 = line.find(',', p2+1);
        
        string id = line.substr(0, p1);
        string xs = line.substr(p1+1, p2-p1-1);
        string ys = line.substr(p2+1, p3-p2-1);
        string ds = line.substr(p3+1);
        
        // Remove 's' prefix if present
        if (xs[0] == 's') xs = xs.substr(1);
        if (ys[0] == 's') ys = ys.substr(1);
        if (ds[0] == 's') ds = ds.substr(1);
        
        // Parse id: NNN_idx
        int n = stoi(id.substr(0, 3));
        int idx = stoi(id.substr(4));
        
        configs[n].x[idx] = stold(xs);
        configs[n].y[idx] = stold(ys);
        configs[n].a[idx] = stold(ds);
    }
    
    for (int n = 1; n <= MAX_N; n++) {
        configs[n].updAll();
        best_sides[n] = configs[n].side();
    }
}

void save_csv(const string& filename) {
    ofstream f(filename);
    f << fixed << setprecision(15);
    f << "id,x,y,deg\\n";
    
    for (int n = 1; n <= MAX_N; n++) {
        for (int i = 0; i < n; i++) {
            f << setfill('0') << setw(3) << n << "_" << i << ",";
            f << "s" << configs[n].x[i] << ",";
            f << "s" << configs[n].y[i] << ",";
            f << "s" << configs[n].a[i] << "\\n";
        }
    }
}

long double calc_total_score() {
    long double total = 0;
    for (int n = 1; n <= MAX_N; n++) {
        total += best_sides[n] * best_sides[n] / n;
    }
    return total;
}

int main(int argc, char** argv) {
    int n_iters = 1000;
    int n_rounds = 16;
    
    for (int i = 1; i < argc; i++) {
        if (string(argv[i]) == "-n" && i+1 < argc) n_iters = stoi(argv[++i]);
        if (string(argv[i]) == "-r" && i+1 < argc) n_rounds = stoi(argv[++i]);
    }
    
    cout << "Tree Packer Optimizer" << endl;
    cout << "Iterations: " << n_iters << ", Rounds: " << n_rounds << endl;
    
    parse_csv("submission.csv");
    cout << fixed << setprecision(6);
    cout << "Initial: " << calc_total_score() << endl;
    
    #pragma omp parallel for schedule(dynamic)
    for (int n = 1; n <= MAX_N; n++) {
        FastRNG rng(42 + n);
        Cfg best = configs[n];
        long double best_side = best.side();
        
        for (int r = 0; r < n_rounds; r++) {
            Cfg c = best;
            
            // Simulated annealing
            c = simulatedAnnealing(c, n_iters, rng);
            
            // Local search
            c = localSearch(c, 50);
            
            // Compaction
            c = compaction(c, 20);
            
            // Squeeze
            c = squeeze(c);
            
            if (c.side() < best_side && !c.anyOvl()) {
                best_side = c.side();
                best = c;
            }
        }
        
        #pragma omp critical
        {
            if (best_side < best_sides[n]) {
                configs[n] = best;
                best_sides[n] = best_side;
            }
        }
    }
    
    cout << "Final:   " << calc_total_score() << endl;
    save_csv("submission_optimized.csv");
    
    return 0;
}
'''

with open('/home/code/experiments/001_baseline/tree_packer.cpp', 'w') as f:
    f.write(cpp_code)

print("C++ optimizer code written to tree_packer.cpp")

C++ optimizer code written to tree_packer.cpp


In [6]:
# Compile the C++ optimizer
import subprocess
os.chdir('/home/code/experiments/001_baseline')

result = subprocess.run(
    ['g++', '-O3', '-march=native', '-std=c++17', '-fopenmp', '-o', 'tree_packer', 'tree_packer.cpp'],
    capture_output=True, text=True
)

if result.returncode == 0:
    print("Compilation successful!")
else:
    print(f"Compilation failed:\n{result.stderr}")

Compilation successful!


In [7]:
# Copy sample submission to working directory
import shutil
shutil.copy('/home/data/sample_submission.csv', '/home/code/experiments/001_baseline/submission.csv')
print("Copied sample_submission.csv to working directory")

Copied sample_submission.csv to working directory


In [8]:
# Run the optimizer with moderate settings for baseline
import time

start_time = time.time()
result = subprocess.run(
    ['./tree_packer', '-n', '2000', '-r', '32'],
    capture_output=True, text=True,
    cwd='/home/code/experiments/001_baseline'
)

elapsed = time.time() - start_time
print(f"Optimization completed in {elapsed:.1f} seconds")
print(result.stdout)
if result.stderr:
    print(f"Errors: {result.stderr}")

Optimization completed in 290.6 seconds
Tree Packer Optimizer
Iterations: 2000, Rounds: 32
Initial: 173.652299
Final:   146.886893



In [ ]:
# Now apply fix_direction rotation optimization
def calculate_bbox_side_at_angle(angle_deg, points):
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])


def optimize_rotation(trees):
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points) / float(scale_factor)
    
    hull_points = points_np[ConvexHull(points_np).vertices]
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.001, 89.999), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun
    
    improvement = initial_side - found_side
    EPSILON = 1e-8
    
    if improvement > EPSILON:
        return found_side, found_angle_deg
    else:
        return initial_side, 0.0


def apply_rotation(trees, angle_deg):
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]
    
    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0]) / float(scale_factor)
    
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])
    
    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center
    
    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            str(rotated[i, 0]), 
            str(rotated[i, 1]),
            str(float(trees[i].angle) + angle_deg)
        )
        rotated_trees.append(new_tree)
    return rotated_trees

print("Rotation optimization functions defined")

In [ ]:
# Apply fix_direction to the optimized submission
def fix_direction(input_path, output_path):
    df = pd.read_csv(input_path)
    
    # Parse all groups
    dict_of_tree_list = {}
    dict_of_side_length = {}
    
    df['x_clean'] = df['x'].astype(str).str.lstrip('s')
    df['y_clean'] = df['y'].astype(str).str.lstrip('s')
    df['deg_clean'] = df['deg'].astype(str).str.lstrip('s')
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    
    for group_id, group_data in df.groupby('group_id'):
        tree_list = [ChristmasTree(center_x=row['x_clean'], center_y=row['y_clean'], angle=row['deg_clean'])
                     for _, row in group_data.iterrows()]
        dict_of_tree_list[group_id] = tree_list
        
        # Calculate current side length
        xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in tree_list])
        min_x, min_y = xys.min(axis=0)
        max_x, max_y = xys.max(axis=0)
        dict_of_side_length[group_id] = max(max_x - min_x, max_y - min_y)
    
    # Calculate initial score
    initial_score = sum(v**2 / int(k) for k, v in dict_of_side_length.items())
    print(f"Initial score: {initial_score:.6f}")
    
    # Apply rotation optimization to each group
    improved_count = 0
    for group_id in sorted(dict_of_tree_list.keys()):
        trees = dict_of_tree_list[group_id]
        if len(trees) < 2:
            continue
            
        try:
            best_side, best_angle = optimize_rotation(trees)
            if abs(best_angle) > 0.001:
                rotated_trees = apply_rotation(trees, best_angle)
                dict_of_tree_list[group_id] = rotated_trees
                
                # Recalculate side length
                xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in rotated_trees])
                min_x, min_y = xys.min(axis=0)
                max_x, max_y = xys.max(axis=0)
                new_side = max(max_x - min_x, max_y - min_y)
                
                if new_side < dict_of_side_length[group_id]:
                    dict_of_side_length[group_id] = new_side
                    improved_count += 1
        except Exception as e:
            pass  # Skip groups that fail
    
    # Calculate final score
    final_score = sum(v**2 / int(k) for k, v in dict_of_side_length.items())
    print(f"Final score after rotation: {final_score:.6f}")
    print(f"Improved {improved_count} groups")
    
    # Save the result
    rows = []
    for group_id in sorted(dict_of_tree_list.keys(), key=int):
        trees = dict_of_tree_list[group_id]
        for i, tree in enumerate(trees):
            rows.append({
                'id': f"{int(group_id):03d}_{i}",
                'x': f"s{float(tree.center_x)}",
                'y': f"s{float(tree.center_y)}",
                'deg': f"s{float(tree.angle)}"
            })
    
    result_df = pd.DataFrame(rows)
    result_df.to_csv(output_path, index=False)
    print(f"Saved to {output_path}")
    
    return final_score

# Apply fix_direction
optimized_path = '/home/code/experiments/001_baseline/submission_optimized.csv'
final_path = '/home/code/experiments/001_baseline/submission_final.csv'

if os.path.exists(optimized_path):
    final_score = fix_direction(optimized_path, final_path)
else:
    print("Optimized file not found, using original")
    final_score = fix_direction('/home/code/experiments/001_baseline/submission.csv', final_path)

In [ ]:
# Validate the final submission
print("\nValidating final submission...")
final_result = score_and_validate_submission(final_path)
print(f"\nFinal validated score: {final_result['total_score']:.6f}")

In [ ]:
# Copy to submission folder
import shutil
shutil.copy(final_path, '/home/submission/submission.csv')
print("Copied final submission to /home/submission/submission.csv")

# Print summary
print(f"\n=== EXPERIMENT SUMMARY ===")
print(f"Baseline score (sample_submission): {baseline_result['total_score']:.6f}")
print(f"Final optimized score: {final_result['total_score']:.6f}")
print(f"Improvement: {baseline_result['total_score'] - final_result['total_score']:.6f}")
print(f"Target to beat: 68.931058")